In [1]:
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize 
from geopy.geocoders import Nominatim

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium
import re

print('Libraries imported.')

Libraries imported.


In [2]:
html_data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
canada_df = pd.DataFrame(html_data[0])
canada_df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<h3>Cleaning the data

In [3]:
canada_df = canada_df[canada_df.Borough != 'Not assigned']

<h3>Grouping Neighborhoods by ',' instead of a '/'

In [5]:
canada_df['Neighborhood'] = canada_df['Neighborhood'].apply(lambda x : re.sub(' / ',',',x))

<h3>Replacing not assigned neighborhoods with borough names

In [6]:
canada_df['Neighborhood'] = canada_df['Neighborhood'].where((canada_df['Neighborhood'] != 'Not assigned'), canada_df['Borough'])
canada_df.reset_index(drop=True,inplace=True)
canada_df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


<h3> Using a csv file with location information

In [7]:
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.set_index(['Postal Code'], inplace=True)
canada_df.set_index(['Postal code'], inplace=True)
canada_lat_long = canada_df.join(coord,how='left')
canada_lat_long.reset_index(inplace=True)
canada_lat_long.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.662301,-79.389494


<h3>Let's look at all the neighborhoods in Canada on the map

In [8]:
locator = Nominatim(user_agent="myAgent", timeout=10)
location = locator.geocode('Toronto, Canada')


canada_map = folium.Map([location.latitude,location.longitude],zoom_start=11)
for neigh, lat,long in zip(canada_lat_long['Neighborhood'],canada_lat_long['Latitude'],canada_lat_long['Longitude']):
    label = folium.Popup(neigh, parse_html=True)
    folium.CircleMarker([lat,long], popup=label, parse_html=True).add_to(canada_map)

canada_map

<h3> Initializing data for the Foursquare API

In [9]:
CLIENT_ID = 'GMINQGQGTEJRL5PV23OJH45IFASRCIYGCBKOJICSTLV431MH' # your Foursquare ID
CLIENT_SECRET = 'LL0FGH4MLG4BLKLCYVGPRDCJBQVSLE2DVO21HPKZCZ4A5LQL' # your Foursquare Secret
VERSION = '20200430'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GMINQGQGTEJRL5PV23OJH45IFASRCIYGCBKOJICSTLV431MH
CLIENT_SECRET:LL0FGH4MLG4BLKLCYVGPRDCJBQVSLE2DVO21HPKZCZ4A5LQL


In [14]:
def getItalianRestaurants(names, latitudes, longitudes):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                'Italian Restaurant', 
                LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [15]:
cannada_venues = getItalianRestaurants(names = canada_lat_long['Neighborhood'], latitudes=
                                 canada_lat_long['Latitude'], longitudes=canada_lat_long['Longitude'])
cannada_venues.head()

KeyError: 'groups'